In [15]:
import json
import pandas as pd
from datasets import Dataset
from bs4 import BeautifulSoup
from tqdm import tqdm

In [16]:
data = []
with open('hf-datasets/raw-datasets/lowyat.jsonl') as fopen:
    for l in tqdm(fopen):
        data.append(json.loads(l))

17323317it [00:43, 398274.34it/s]


In [17]:
len(data)

17323317

In [18]:
data_dict = {
    'text': data
}

In [19]:
dataset = Dataset.from_dict(data_dict)

In [20]:
dataset

Dataset({
    features: ['text'],
    num_rows: 17323317
})

In [21]:
dataset.save_to_disk("hf-datasets/raw-datasets/lowyat")

Saving the dataset (0/12 shards):   0%|          | 0/17323317 [00:00<?, ? examples/s]

### 2. Run below command

make sure directory for the script is correct before running the command

- path -  your dataset to dedupe path
- name - relevant only when u use dataset directly from huggingface
- output - output directory of the dedupe dataset
- column - column of the text we are going to remove duplicate
- threshold - jaccard similarity threshold
- local - (local dataset or directly from huggingface) remove if dataset is not local

In [29]:
command = f"python3 -m text_dedup.minhash \
  --path hf-datasets/raw-datasets/lowyat \
  --split train \
  --cache_dir ./cache \
  --output hf-datasets/dedupe-datasets/lowyat \
  --column text \
  --batch_size 10000 \
  --threshold 0.95 \
  --min_length 1 \
  --local"

command

'python3 -m text_dedup.minhash   --path hf-datasets/raw-datasets/lowyat   --split train   --cache_dir ./cache   --output hf-datasets/dedupe-datasets/lowyat   --column text   --batch_size 10000   --threshold 0.95   --min_length 1   --local'

In [1]:
import subprocess
subprocess.run(command, shell=True)

The script produced the following

- duplicates.csv : containing duplicate pairs indices (document index)
- duplicate_cluster.csv: the text of duplicate pairs for view
- the deduped dataset in huggingface dataset type (from output argument)
- dedup dataset in jsonl format

In [31]:
!wc -l hf-datasets/dedupe-datasets/lowyat.jsonl

17098395 hf-datasets/dedupe-datasets/lowyat.jsonl


In [33]:
!ls -lh hf-datasets/dedupe-datasets/lowyat.jsonl

-rw-r--r-- 1 ubuntu ubuntu 5.7G Aug 18 02:27 hf-datasets/dedupe-datasets/lowyat.jsonl


In [34]:
!rm -rf hf-datasets/dedupe-datasets/lowyat hf-datasets/raw-datasets/lowyat